In [22]:
#FOR PART I
import sys
import numpy as np
from functools import lru_cache,wraps
import collections as col
import string as st
import re
import time
import copy 

sys.setrecursionlimit(15000) 

def Read_file(file_name):
    Beacons=[]
    with open(file_name) as input_file:
        list_of_lines=input_file.read().splitlines()
    Valves=[]
    for line in list_of_lines:
        line=line.replace('Valve ','')
        line=line.replace(' has flow rate=',';')
        line=line.replace('; tunnels lead to valves ',';')
        line=line.replace('; tunnels lead to valve ',';')
        line=line.replace('; tunnel leads to valve ',';')
        line=line.replace('; tunnel leads to valves ',';')
        line=line.replace(' ','')
        Valves.append(line.split(';'))
    return Valves

#         Beacons.append([int(i) for i in line.split(',')])
#     return Beacons



def Update(valve,uselessvalve):
    for destination in Tunnels[uselessvalve]:
        if (destination in Tunnels[valve]):
            Tunnels[valve][destination]=min(Tunnels[valve][destination],Tunnels[valve][uselessvalve]+Tunnels[uselessvalve][destination])
        elif (destination!=valve):
            Tunnels[valve][destination]=Tunnels[valve][uselessvalve]+Tunnels[uselessvalve][destination]
            

def Heuristic(time,score,opened,shift):
    gen=(x for x in opened if opened[x]==0)
    return sum([Flow[node] for node in gen])*(30-time-shift)+score

def Heuristic2(time,score,opened,shift):
    gen=(x for x in opened if opened[x]==0)
    Flows=sorted([Flow[node] for node in gen],reverse=True)
    temp=0
    for i in range(len(Flows)):
        temp+=Flows[i]*(30-time-2*i-shift)
    return temp+score

# @lru_cache(maxsize=20000)  
def Score(node,time,score,opened):
    global MaxScore
    global N

    #print(node,time,score,opened)
    if(time>MaxTime):
        #print('time',score)
        MaxScore=max(score,MaxScore)
        N+=1;
        if(N%100000==0):
            print(N,MaxScore)
        return score
    elif(0 not in opened.values()):
        #print('opened',score)
        MaxScore=max(score,MaxScore)
        N+=1;
        if(N%100000==0):
            print(N,MaxScore)
        return score
    elif(opened[node]==0):
        if(Heuristic2(time,score,opened,1)<MaxScore):
            N+=1;
            if(N%100000==0):
                print(N,MaxScore)
            return MaxScore
        else:
            score1=max([Score(target,time+Tunnels[node][target],score,opened) for target in Tunnels[node]])
            opened2=copy.deepcopy(opened)
            opened2[node]=1
            time+=1
            score2=max([Score(target,time+Tunnels[node][target],score+Flow[node]*(30-time),opened2) for target in Tunnels[node]])
            return max(score1,score2)   
    else:
        if(Heuristic2(time,score,opened,2)<MaxScore):
            N+=1;
            if(N%100000==0):
                print(N,MaxScore)
            return MaxScore
        else:
            return max([Score(target,time+Tunnels[node][target],score,opened) for target in Tunnels[node]])
        
            

if __name__ == '__main__':
    N=0
    Valves=Read_file('example')
#     Valves=Read_file('input')
#     print(Valves)
    Flow={}
    Tunnels={}
    Open={}
    MaxTime=30
    for valve in Valves:
        #Useless valves considered open
        #Flow store the flow of the valve
        Flow[valve[0]]=eval(valve[1])
        Open[valve[0]]=0
        #Tunnels store the valves which can be accessed from valve
        Tunnels[valve[0]]={ Valve : 1 for Valve in valve[2].split(',') }
    #print(Flow)
    #print(Tunnels)
    MaxScore=0
    Time=0
    Starting="AA"
    Open[Starting]=1
    gen = (valve for valve in Flow if Flow[valve]==0 and valve!=Starting)
    #All these valves are simply longer tunnels
    for uselessvalve in list(gen):
        #Look through all valves
        for valve in list(Tunnels):
            #if this valve leads to a tunnel
            if (uselessvalve in Tunnels[valve]):
                print(Tunnels[valve])
                print(Tunnels[uselessvalve])
                Update(valve,uselessvalve)
                print(Tunnels[valve])
                print('')
                Tunnels[valve].pop(uselessvalve, None)
        Tunnels.pop(uselessvalve, None)
        Open.pop(uselessvalve,None)
        Flow.pop(uselessvalve,None)
    for valve in Tunnels:
        Tunnels[valve].pop(valve,None)
    print('')

    print('')
    print(Flow)
    print(Tunnels)
    gen = (valve for valve in Flow if Flow[valve]==0 and valve!=Starting)


    print('')
    print('')
    print(Flow)
    print(Tunnels)
    print(Open)
#                 print(uselessvalve,valve,Tunnels[valve])
    
    print(Score('AA',0,0,Open))
    

{'FF': 1, 'DD': 1}
{'EE': 1, 'GG': 1}
{'FF': 1, 'DD': 1, 'GG': 2}

{'FF': 1, 'HH': 1}
{'EE': 1, 'GG': 1}
{'FF': 1, 'HH': 1, 'EE': 2}

{'DD': 1, 'GG': 2}
{'HH': 1, 'EE': 2}
{'DD': 1, 'GG': 2, 'HH': 3}

{'GG': 1}
{'HH': 1, 'EE': 2}
{'GG': 1, 'EE': 3}

{'DD': 1, 'II': 1, 'BB': 1}
{'AA': 1, 'JJ': 1}
{'DD': 1, 'II': 1, 'BB': 1, 'JJ': 2}

{'II': 1}
{'AA': 1, 'JJ': 1}
{'II': 1, 'AA': 2}



{'AA': 0, 'BB': 13, 'CC': 2, 'DD': 20, 'EE': 3, 'HH': 22, 'JJ': 21}
{'AA': {'DD': 1, 'BB': 1, 'JJ': 2}, 'BB': {'CC': 1, 'AA': 1}, 'CC': {'DD': 1, 'BB': 1}, 'DD': {'CC': 1, 'AA': 1, 'EE': 1}, 'EE': {'DD': 1, 'HH': 3}, 'HH': {'EE': 3}, 'JJ': {'AA': 2}}


{'AA': 0, 'BB': 13, 'CC': 2, 'DD': 20, 'EE': 3, 'HH': 22, 'JJ': 21}
{'AA': {'DD': 1, 'BB': 1, 'JJ': 2}, 'BB': {'CC': 1, 'AA': 1}, 'CC': {'DD': 1, 'BB': 1}, 'DD': {'CC': 1, 'AA': 1, 'EE': 1}, 'EE': {'DD': 1, 'HH': 3}, 'HH': {'EE': 3}, 'JJ': {'AA': 2}}
{'AA': 1, 'BB': 0, 'CC': 0, 'DD': 0, 'EE': 0, 'HH': 0, 'JJ': 0}
100000 1047
200000 1489
1651


In [24]:
#FOR PART II
import sys
import numpy as np
from functools import lru_cache,wraps
import collections as col
import string as st
import re
import time
import copy 

sys.setrecursionlimit(15000) 

def Read_file(file_name):
    Beacons=[]
    with open(file_name) as input_file:
        list_of_lines=input_file.read().splitlines()
    Valves=[]
    for line in list_of_lines:
        line=line.replace('Valve ','')
        line=line.replace(' has flow rate=',';')
        line=line.replace('; tunnels lead to valves ',';')
        line=line.replace('; tunnels lead to valve ',';')
        line=line.replace('; tunnel leads to valve ',';')
        line=line.replace('; tunnel leads to valves ',';')
        line=line.replace(' ','')
        Valves.append(line.split(';'))
    return Valves

#         Beacons.append([int(i) for i in line.split(',')])
#     return Beacons



def Update(valve,uselessvalve):
    for destination in Tunnels[uselessvalve]:
        if (destination in Tunnels[valve]):
            Tunnels[valve][destination]=min(Tunnels[valve][destination],Tunnels[valve][uselessvalve]+Tunnels[uselessvalve][destination])
        elif (destination!=valve):
            Tunnels[valve][destination]=Tunnels[valve][uselessvalve]+Tunnels[uselessvalve][destination]
            

def Heuristic(time,score,opened,shift):
    gen=(x for x in opened if opened[x]==0)
    return sum([Flow[node] for node in gen])*(30-time-shift)+score

def Heuristic2(time,score,opened,shift):
    gen=(x for x in opened if opened[x]==0)
    Flows=sorted([Flow[node] for node in gen],reverse=True)
    temp=0
    for i in range(len(Flows)):
        temp+=Flows[i]*(30-time-2*i-shift)
    return temp+score


# @lru_cache(maxsize=20000)  
#This cmompute the score given that the players will arrive at nodes n1 and n2 
#at times t1 and t2 and the time is currently equal to time
def Score(n1,t1,n2,t2,time,score,opened):
    global MaxScore
    global N

    #If the time is over, terminate
    if(time>=MaxTime):
        #print('time',score)
        MaxScore=max(score,MaxScore)
        N+=1;
        if(N%1000000==0):
            print(N,MaxScore)
        return score
    #If all valves are opened, terminate
    elif(0 not in opened.values()):
        #print('opened',score)
        MaxScore=max(score,MaxScore)
        N+=1;
        if(N%1000000==0):
            print(N,MaxScore)
        return score
    elif(time<t1 and time<t2):
        time+=min(t1-time,t2-time);
        return Score(n1,t1,n2,t2,time,score,opened)
    #Else, somebody has arrived and should move
    elif(t1==time):
        #If the valve is opened, move
        if(opened[n1]==1):
            if(Heuristic2(time,score,opened,2)<MaxScore):
                N+=1;
                if(N%1000000==0):
                    print(N,MaxScore)
                return MaxScore
            else:
#                 print([ [target,t1+Tunnels[n1][target],n2,t2,time] for target in Tunnels[n1]])
                return max([Score(target,t1+Tunnels[n1][target],n2,t2,time,score,opened) for target in Tunnels[n1]])
        #If the valve is closed
        else:
            if(Heuristic2(time,score,opened,1)<MaxScore):
                N+=1;
                if(N%1000000==0):
                    print(N,MaxScore)
                return MaxScore
            else:
                #score obtained by moving
#                 print([[target,t1+Tunnels[n1][target],n2,t2,time] for target in Tunnels[n1]])
                score1=max([Score(target,t1+Tunnels[n1][target],n2,t2,time,score,opened) for target in Tunnels[n1]])
                #score obtained by opening
                opened2=copy.deepcopy(opened)
                opened2[n1]=1
                return Score(n1,t1+1,n2,t2,time,score+Flow[n1]*(MaxTime-t1-1),opened2)
    elif(t2==time):
        #If the valve is opened, move
        if(opened[n2]==1):
            if(Heuristic2(time,score,opened,1)<MaxScore):
                N+=1;
                if(N%1000000==0):
                    print(N,MaxScore)
                return MaxScore
            else:
#                 print([[n1,t1,target,t2+Tunnels[n2][target],time] for target in Tunnels[n2]])
                return max([Score(n1,t1,target,t2+Tunnels[n2][target],time,score,opened) for target in Tunnels[n2]])
        #If the valve is closed
        else:
            if(Heuristic2(time,score,opened,1)<MaxScore):
                N+=1;
                if(N%1000000==0):
                    print(N,MaxScore)
                return MaxScore
            else:
                #score obtained by moving
#                 print([[n1,t1,target,t2+Tunnels[n2][target],time] for target in Tunnels[n2]])
                score1=max([Score(n1,t1,target,t2+Tunnels[n2][target],time,score,opened) for target in Tunnels[n2]])
                #score obtained by opening
                opened2=copy.deepcopy(opened)
                opened2[n2]=1
                return Score(n1,t1,n2,t2+1,time,score+Flow[n2]*(MaxTime-t2-1),opened2)

            

if __name__ == '__main__':
    N=0
#     Valves=Read_file('example')
    Valves=Read_file('input')
    Flow={}
    Tunnels={}
    Open={}
    MaxTime=26
    for valve in Valves:
        #Useless valves considered open
        #Flow store the flow of the valve
        Flow[valve[0]]=eval(valve[1])
        Open[valve[0]]=0
        #Tunnels store the valves which can be accessed from valve
        Tunnels[valve[0]]={ Valve : 1 for Valve in valve[2].split(',') }
    MaxScore=0
    Time=0
    Starting="AA"
    Open[Starting]=1
    gen = (valve for valve in Flow if Flow[valve]==0 and valve!=Starting)
    #All these valves are simply longer tunnels
    for uselessvalve in list(gen):
        #Look through all valves
        for valve in list(Tunnels):
            #if this valve leads to a tunnel
            if (uselessvalve in Tunnels[valve]):
                Update(valve,uselessvalve)
                Tunnels[valve].pop(uselessvalve, None)
        Tunnels.pop(uselessvalve, None)
        Open.pop(uselessvalve,None)
        Flow.pop(uselessvalve,None)
    for valve in Tunnels:
        Tunnels[valve].pop(valve,None)
    print('')
    print(Flow)
    print(Tunnels)
    print(Open)
    
    print(Score('AA',0,'AA',0,0,0,Open))
    


{'KV': 13, 'SU': 19, 'RI': 4, 'VT': 5, 'HR': 11, 'QF': 17, 'UH': 25, 'RP': 3, 'GG': 12, 'YA': 21, 'LI': 15, 'DT': 16, 'AA': 0, 'CN': 7, 'JQ': 6, 'NS': 23}
{'KV': {'HR': 3, 'RI': 2, 'NS': 2, 'DT': 3, 'GG': 3}, 'SU': {'GG': 2, 'DT': 3, 'JQ': 3, 'CN': 3}, 'RI': {'AA': 2, 'VT': 2, 'KV': 2, 'RP': 3, 'GG': 2}, 'VT': {'RI': 2, 'JQ': 3, 'RP': 2, 'CN': 3, 'AA': 3}, 'HR': {'NS': 2, 'KV': 3, 'YA': 3}, 'QF': {'JQ': 3, 'UH': 2}, 'UH': {'QF': 2}, 'RP': {'VT': 2, 'RI': 3, 'AA': 3, 'CN': 3, 'GG': 3}, 'GG': {'SU': 2, 'RI': 2, 'DT': 2, 'KV': 3, 'RP': 3}, 'YA': {'NS': 2, 'HR': 3}, 'LI': {'DT': 3}, 'DT': {'LI': 3, 'SU': 3, 'KV': 3, 'GG': 2}, 'AA': {'RI': 2, 'RP': 3, 'JQ': 3, 'CN': 3, 'VT': 3}, 'CN': {'SU': 3, 'AA': 3, 'VT': 3, 'JQ': 2, 'RP': 3}, 'JQ': {'VT': 3, 'SU': 3, 'AA': 3, 'QF': 3, 'CN': 2}, 'NS': {'HR': 2, 'KV': 2, 'YA': 2}}
{'KV': 0, 'SU': 0, 'RI': 0, 'VT': 0, 'HR': 0, 'QF': 0, 'UH': 0, 'RP': 0, 'GG': 0, 'YA': 0, 'LI': 0, 'DT': 0, 'AA': 1, 'CN': 0, 'JQ': 0, 'NS': 0}
1000000 1051
2000000 1074
3000